# ML testing: experiment #3f- harmonized with CovBat

This notebook involves testing for the clinical harmonization paper. This notebook shows mixed_dataset (TOP + StrokeMRI) based models with CovBat harmonization to Insight 46 and SABRE datasets (the strokeMRI and TOP become one dataset)

Harmonisation: CovBat

Training data: NORMENT which is StrokeMRI and TOP togehter

Testing data: test set from NORMENT

Futher data applied to: SABRE, Insight46, EDIS, (HELIUS pending)

Validation method: K-fold, double-stratified

Brain-age algorithms: LR, lasso, extra trees, elasticCV net fully tested (but not optimized parameters), additionals partly

Outputs: SubjectID, real age, predicted age of validation and testing sets

### import libraries

In [10]:
import os       # using operating system dependent functionality (folders)
import sys

import glob
from functools import reduce

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt

# # demo stuff
# import ipywidgets as widgets
# import seaborn 

# ml stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score

import joblib

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

### import data

In [11]:
filepath_mri_for_ids = '../open_work/internal_results/cleaned_pvc2s' 
filename_mri_for_ids = os.path.join(filepath_mri_for_ids,'StrokeMRI_pvc2c.csv') 

filepath_top_for_ids = '../open_work/internal_results/cleaned_pvc2s/' 
filename_top_for_ids = os.path.join(filepath_top_for_ids,'TOP_pvc2c.csv') 

filepath_topmri =  'harmonizations/harm_results/covbat/'  
filename_topmri = os.path.join(filepath_topmri,'topmri_vs_sabre_5way.csv') 

filepath_sabre = 'harmonizations/harm_results/covbat/' 
filename_sabre = os.path.join(filepath_topmri,'sabre_vs_topmri_5way.csv') 

filepath_insight46 = 'harmonizations/harm_results/covbat/' 
filename_insight46 =  os.path.join(filepath_topmri,'insight_vs_topmri_5way.csv') 

# read in data

TOPMRI = pd.read_csv(filename_topmri)
SABRE = pd.read_csv(filename_sabre)
Insight46 = pd.read_csv(filename_insight46)
# take extra column off
TOPMRI = TOPMRI.drop(TOPMRI.columns[0],axis=1)
SABRE = SABRE.drop(SABRE.columns[0],axis=1)
#StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)
Insight46 = Insight46.drop(Insight46.columns[0],axis=1)
IDS_TOP =  pd.read_csv(filename_top_for_ids)
IDS_MRI =  pd.read_csv(filename_mri_for_ids)

FileNotFoundError: [Errno 2] No such file or directory: 'harmonizations/harm_results/covbat/topmri_vs_sabre_5way.csv'

In [12]:
#SABRE

In [13]:
# # Now we need to flip the sex back to numbers for a correlation
# sex_mapping = {'F':0,'M':1}
# TOPMRI = TOPMRI.assign(sex = TOPMRI.sex.map(sex_mapping))
# TOPMRI.head(3)

In [14]:
# Insight46 = Insight46.assign(sex = Insight46.sex.map(sex_mapping))
# #Insight46.head(3)

In [15]:
coly = TOPMRI.columns
#SABRE = SABRE.assign(sex = SABRE.sex.map(sex_mapping))
SABRE = SABRE[coly]
SABRE.head(3)

NameError: name 'TOPMRI' is not defined

In [16]:
SABRE =SABRE.dropna()
#SABRE.isna().sum()

NameError: name 'SABRE' is not defined

# Now we need to break up the TOP and StrokeMRI datasets as well, and format them like the others

In [17]:
set_top_ids = set(IDS_TOP.participant_id)
set_mri_ids = set(IDS_MRI.participant_id)
StrokeMRI = TOPMRI[TOPMRI['participant_id'].isin(list(set_mri_ids))]
TOP = TOPMRI[TOPMRI['participant_id'].isin(list(set_top_ids))]
TOP.head(3)

NameError: name 'IDS_TOP' is not defined

In [ ]:
# StrokeMRI = StrokeMRI.assign(sex = StrokeMRI.sex.map(sex_mapping))
# StrokeMRI.head(3)

In [13]:
StrokeMRI

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf


In [14]:
#TOP = TOP.assign(sex = TOP.sex.map(sex_mapping))
TOP.head(3)

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf


In [15]:
# check for any duplicated patients between stroke and mri
strokers = set(StrokeMRI.participant_id)
topers = set(TOP.participant_id)
z = strokers.intersection(topers)
print(z) 

set()


In [16]:
# make mixed StrokeMRI and TOP dataset
#mixed_data = pd.concat([TOP, StrokeMRI], sort=False)
mixed_data = TOPMRI

In [ ]:
output_folder = '3_CovBat'

os.makedirs(output_folder, exist_ok=True)

## Build ML models

In [17]:
ml_matrix = mixed_data #.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
y = ml_matrix['age'].values
y=y.astype('float')

In [18]:
linr_k_frame, linr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('linear regression', 'req_covbatharm_mix_linr', LinearRegression(), ml_matrix, X[:,1:], y)

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

In [19]:
linr_k_frame

,algorithm,fold,file_name,mae,r2,explained_variance
0,linear regression-0,0,req_neuroharm_mix_linr.0,4.067082,0.887160,0.889995
0,linear regression-1,1,req_neuroharm_mix_linr.1,3.994878,0.910099,0.910281
0,linear regression-2,2,req_neuroharm_mix_linr.2,4.153087,0.893788,0.894245
0,linear regression-3,3,req_neuroharm_mix_linr.3,4.118220,0.871758,0.872410
0,linear regression-4,4,req_neuroharm_mix_linr.4,4.090613,0.895278,0.895281


In [20]:
linr_k_frame.to_csv(output_folder + 'linr_k_frame_covbat_harm.csv')

In [21]:
avg_linr = sep.avg_k_folds(linr_k_frame)
avg_linr

,algorithm,file_name,mae,r2,explained_variance
0,0 linear regression-0 0 linear regressio...,0 req_neuroharm_mix_linr.0 0 req_neuroha...,4.084776,0.891617,0.892442


In [22]:
linr_y_frame.to_csv(output_folder + 'linr_y_frame_covbat_harm.csv')
linr_y_frame

,y_test,y_pred
0,27.550000,26.772186
1,52.495890,56.051926
2,31.372603,34.416129
3,50.345205,49.160239
4,77.593735,73.311741
...,...,...
272,69.424658,80.279743
273,48.730000,46.476787
274,60.183060,64.419847
275,40.355191,38.041194


In [23]:
linr_y_frame.to_csv(output_folder + 'linr_y_frame_covbat_harm.csv')

In [24]:
linr = models[0]
linr[0]

LinearRegression()

In [25]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [26]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'unharm_mix_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'unharm_mix_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'unharm_mix_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'unharm_mix_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'unharm_mix_linr4.sav'))

In [27]:
llreg_k_frame, llreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('lasso regression', 'unharm_mix_llreg',  linear_model.LassoLars(alpha=0.01), ml_matrix, X[:,1:], y)
llreg_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,lasso regression-0,0,unharm_mix_llreg.0,4.873434,0.839136,0.841900
0,lasso regression-1,1,unharm_mix_llreg.1,4.722047,0.874001,0.874133
0,lasso regression-2,2,unharm_mix_llreg.2,4.972792,0.865074,0.866453
0,lasso regression-3,3,unharm_mix_llreg.3,5.200974,0.795080,0.795934
0,lasso regression-4,4,unharm_mix_llreg.4,5.106406,0.849541,0.850091


In [28]:
llreg_k_frame.to_csv(output_folder + 'llreg_k_frame_covbat_harm.csv')

In [29]:
avg_llreg = sep.avg_k_folds(llreg_k_frame)
avg_llreg

,algorithm,file_name,mae,r2,explained_variance
0,0 lasso regression-0 0 lasso regression-...,0 unharm_mix_llreg.0 0 unharm_mix_llreg....,4.97513,0.844566,0.845702


In [30]:
llreg_y_frame

,y_test,y_pred
0,27.550000,40.151253
1,52.495890,55.252114
2,31.372603,45.407681
3,50.345205,56.669340
4,77.593735,66.522655
...,...,...
272,69.424658,84.791350
273,48.730000,46.743489
274,60.183060,57.988285
275,40.355191,38.266613


In [31]:
llreg_y_frame.to_csv(output_folder + 'llreg_y_frame_covbat_harm.csv')

In [32]:
llreg = models[0]
llreg[0]

LassoLars(alpha=0.01)

In [33]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'unharm_mix_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'unharm_mix_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'unharm_mix_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'unharm_mix_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'unharm_mix_linr4.sav'))

In [34]:
dtree_k_frame, dtree_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('decision tree', 'unharm_mix_dtree',  tree.DecisionTreeRegressor(), ml_matrix, X[:,1:], y)
dtree_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,decision tree-0,0,unharm_mix_dtree.0,5.157169,0.844841,0.844975
0,decision tree-1,1,unharm_mix_dtree.1,5.047306,0.855028,0.855622
0,decision tree-2,2,unharm_mix_dtree.2,5.852512,0.791690,0.791738
0,decision tree-3,3,unharm_mix_dtree.3,5.597473,0.818179,0.818625
0,decision tree-4,4,unharm_mix_dtree.4,5.430559,0.813725,0.813913


In [35]:
dtree_k_frame.to_csv(output_folder + 'dtree_k_frame_covbat_harm.csv')

In [36]:
avg_dtree = sep.avg_k_folds(dtree_k_frame)
avg_dtree

,algorithm,file_name,mae,r2,explained_variance
0,0 decision tree-0 0 decision tree-1 0 ...,0 unharm_mix_dtree.0 0 unharm_mix_dtree....,5.417004,0.824693,0.824975


In [37]:
dtree_y_frame

,y_test,y_pred
0,27.550000,33.640000
1,52.495890,50.180822
2,31.372603,28.100000
3,50.345205,48.730000
4,77.593735,54.956164
...,...,...
272,69.424658,75.106849
273,48.730000,45.660000
274,60.183060,72.762295
275,40.355191,44.280000


In [38]:
dtree_y_frame.to_csv(output_folder + 'dtree_y_frame_covbat_harm.csv')

In [39]:
dtree = models[0]
dtree[0]

DecisionTreeRegressor()

In [40]:
regr_k_frame, regr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('MLP regression', 'unharm_mix_regr',   MLPRegressor(random_state=1, max_iter=700), ml_matrix, X[:,1:], y)
regr_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,MLP regression-0,0,unharm_mix_regr.0,5.055966,0.859252,0.860574
0,MLP regression-1,1,unharm_mix_regr.1,5.543250,0.821933,0.842951
0,MLP regression-2,2,unharm_mix_regr.2,5.147595,0.835158,0.835197
0,MLP regression-3,3,unharm_mix_regr.3,6.213328,0.781484,0.796718
0,MLP regression-4,4,unharm_mix_regr.4,5.941447,0.790803,0.795556


In [41]:
regr_k_frame.to_csv(output_folder + 'regr_k_frame_covbat_harm.csv')

In [42]:
avg_regr = sep.avg_k_folds(regr_k_frame)
avg_regr

,algorithm,file_name,mae,r2,explained_variance
0,0 MLP regression-0 0 MLP regression-1 0 ...,0 unharm_mix_regr.0 0 unharm_mix_regr.1 ...,5.580317,0.817726,0.826199


In [43]:
regr_y_frame

,y_test,y_pred
0,27.550000,37.993776
1,52.495890,54.268943
2,31.372603,38.658520
3,50.345205,54.776673
4,77.593735,60.265754
...,...,...
272,69.424658,78.146877
273,48.730000,51.160823
274,60.183060,52.174720
275,40.355191,42.272936


In [44]:
regr_y_frame.to_csv(output_folder + 'regr_y_frame_covbat_harm.csv')

In [45]:
regr = models[0]
regr[0]

MLPRegressor(max_iter=700, random_state=1)

In [46]:
svrp2_k_frame, svrp2_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('support vector reg poly2', 'unharm_mix_svrp2',   SVR(C=1.0, kernel='poly', degree =2, epsilon=0.2), ml_matrix, X[:,1:], y)
svrp2_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,support vector reg poly2-0,0,unharm_mix_svrp2.0,11.574108,0.335783,0.343499
0,support vector reg poly2-1,1,unharm_mix_svrp2.1,12.163466,0.228923,0.234776
0,support vector reg poly2-2,2,unharm_mix_svrp2.2,11.451623,0.303930,0.315578
0,support vector reg poly2-3,3,unharm_mix_svrp2.3,11.883448,0.298054,0.301903
0,support vector reg poly2-4,4,unharm_mix_svrp2.4,11.726831,0.303163,0.306888


In [47]:
svrp2_k_frame.to_csv(output_folder + 'svrp2_k_frame_covbat_harm.csv')

In [48]:
avg_svrp2 = sep.avg_k_folds(svrp2_k_frame)
avg_svrp2

,algorithm,file_name,mae,r2,explained_variance
0,0 support vector reg poly2-0 0 support v...,0 unharm_mix_svrp2.0 0 unharm_mix_svrp2....,11.759895,0.29397,0.300529


In [49]:
svrp2_y_frame

,y_test,y_pred
0,27.550000,46.424701
1,52.495890,51.031701
2,31.372603,43.691165
3,50.345205,52.574484
4,77.593735,54.566997
...,...,...
272,69.424658,62.428333
273,48.730000,48.424802
274,60.183060,48.504592
275,40.355191,52.497514


In [50]:
svrp2_y_frame.to_csv(output_folder + 'svrp2_y_frame_covbat_harm.csv')

In [51]:
svrp2 = models[0]
svrp2[0]

SVR(degree=2, epsilon=0.2, kernel='poly')

In [52]:
eregr_k_frame, eregr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('elasticnetCV', 'unharm_mix_eregr',  ElasticNetCV(cv=5, random_state=12), ml_matrix, X[:,1:], y)
eregr_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,elasticnetCV-0,0,unharm_mix_eregr.0,9.994718,0.482144,0.483690
0,elasticnetCV-1,1,unharm_mix_eregr.1,10.880828,0.373580,0.373622
0,elasticnetCV-2,2,unharm_mix_eregr.2,9.715762,0.479135,0.482453
0,elasticnetCV-3,3,unharm_mix_eregr.3,10.664553,0.427034,0.427035
0,elasticnetCV-4,4,unharm_mix_eregr.4,10.356312,0.439442,0.439444


In [53]:
eregr_k_frame.to_csv(output_folder + 'eregr_k_frame_covbat_harm.csv')

In [54]:
avg_eregr = sep.avg_k_folds(eregr_k_frame)
avg_eregr

,algorithm,file_name,mae,r2,explained_variance
0,0 elasticnetCV-0 0 elasticnetCV-1 0 e...,0 unharm_mix_eregr.0 0 unharm_mix_eregr....,10.322435,0.440267,0.441249


In [55]:
eregr_y_frame

,y_test,y_pred
0,27.550000,43.509790
1,52.495890,52.724634
2,31.372603,42.667449
3,50.345205,55.938585
4,77.593735,59.821511
...,...,...
272,69.424658,68.821661
273,48.730000,49.302987
274,60.183060,49.636286
275,40.355191,53.169640


In [56]:
eregr_y_frame.to_csv(output_folder + 'eregr_y_frame_covbat_harm.csv')

In [57]:
eregr = models[0]
eregr[0]

ElasticNetCV(cv=5, random_state=12)

In [58]:
etreg_k_frame, etreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('extra trees', 'unharm_mix_etreg',  ExtraTreesRegressor(n_estimators=100, random_state=0), ml_matrix, X[:,1:], y)
etreg_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,extra trees-0,0,unharm_mix_etreg.0,3.784416,0.919928,0.919952
0,extra trees-1,1,unharm_mix_etreg.1,3.611236,0.927018,0.927275
0,extra trees-2,2,unharm_mix_etreg.2,3.541452,0.926866,0.927056
0,extra trees-3,3,unharm_mix_etreg.3,3.806124,0.918779,0.919018
0,extra trees-4,4,unharm_mix_etreg.4,3.963503,0.907399,0.907525


In [59]:
etreg_k_frame.to_csv(output_folder + 'etreg_k_frame_covbat_harm.csv')

In [60]:
avg_etreg = sep.avg_k_folds(etreg_k_frame)
avg_etreg

,algorithm,file_name,mae,r2,explained_variance
0,0 extra trees-0 0 extra trees-1 0 ext...,0 unharm_mix_etreg.0 0 unharm_mix_etreg....,3.741346,0.919998,0.920165


In [61]:
etreg_y_frame

,y_test,y_pred
0,27.550000,36.436874
1,52.495890,54.326182
2,31.372603,39.423492
3,50.345205,50.436353
4,77.593735,67.870246
...,...,...
272,69.424658,72.026577
273,48.730000,44.653897
274,60.183060,66.004888
275,40.355191,35.238613


In [62]:
etreg_y_frame.to_csv(output_folder + 'etreg_y_frame_covbat_harm.csv')

In [63]:
etreg = models[0]
etreg[0]

ExtraTreesRegressor(random_state=0)

In [64]:
## optional save models
#joblib.dump(etreg[0], ('../result_models/'+ 'unharm_mix_etreg0.sav'))
#joblib.dump(etreg[1], ('../result_models/'+ 'unharm_mix_etreg1.sav'))
#joblib.dump(etreg[2], ('../result_models/'+ 'unharm_mix_etreg2.sav'))
#joblib.dump(etreg[3], ('../result_models/'+ 'unharm_mix_etreg3.sav'))
#joblib.dump(etreg[4], ('../result_models/'+ 'unharm_mix_etreg4.sav'))

Show results ON AVERAGE for each model

In [65]:
mixed_based_covbat_harmonized_on_testmix =pd.concat([avg_linr,
                   avg_llreg,
                   avg_dtree,
                   avg_regr,
                   avg_svrp2,
                   avg_eregr,
                   avg_etreg],
                  axis=0)
mixed_based_covbat_harmonized_on_testmix

,algorithm,file_name,mae,r2,explained_variance
0,0 linear regression-0 0 linear regressio...,0 req_neuroharm_mix_linr.0 0 req_neuroha...,4.084776,0.891617,0.892442
0,0 lasso regression-0 0 lasso regression-...,0 unharm_mix_llreg.0 0 unharm_mix_llreg....,4.975130,0.844566,0.845702
0,0 decision tree-0 0 decision tree-1 0 ...,0 unharm_mix_dtree.0 0 unharm_mix_dtree....,5.417004,0.824693,0.824975
0,0 MLP regression-0 0 MLP regression-1 0 ...,0 unharm_mix_regr.0 0 unharm_mix_regr.1 ...,5.580317,0.817726,0.826199
0,0 support vector reg poly2-0 0 support v...,0 unharm_mix_svrp2.0 0 unharm_mix_svrp2....,11.759895,0.293970,0.300529
0,0 elasticnetCV-0 0 elasticnetCV-1 0 e...,0 unharm_mix_eregr.0 0 unharm_mix_eregr....,10.322435,0.440267,0.441249
0,0 extra trees-0 0 extra trees-1 0 ext...,0 unharm_mix_etreg.0 0 unharm_mix_etreg....,3.741346,0.919998,0.920165


In [66]:
mixed_based_covbat_harmonized_on_testmix.to_csv(output_folder + 'mixed_based_covbat_harmonized_on_testmix.csv')

# Running mixed model over SABRE dataset

## Here we will do an example of running  models made of the entire StrokeMRI and TOP dataset mixed as one

#### Build new models

In [67]:
TOPMRI.head(3)

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,...,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf,binned,fuse_bin
0,sub-0668_1,50.40,1,0.617501,0.470392,0.236052,0.463182,0.819954,-0.005404,12.732741,...,0.558961,0.82831,0.719521,18.921497,87.011810,80.241341,55.807275,70.977497,2,6
1,sub-0532_1,37.02,1,0.693920,0.579498,0.294685,0.439796,0.811614,-0.006151,19.612609,...,0.862546,0.60858,0.716976,18.218930,98.415147,90.024820,66.535156,79.219991,1,5
2,sub-0529_1,30.57,1,0.641051,0.444492,0.265101,0.473141,0.796567,-0.002403,15.470687,...,0.620556,0.53582,0.596405,20.082497,106.272833,99.973628,71.505620,86.542509,0,4


In [9]:
ml_matrix = TOPMRI.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')

NameError: name 'TOPMRI' is not defined

In [69]:
# now we decide to do no test train split, rather take all
X_train = X
y_train = y

In [8]:
MIXlinr = LinearRegression()
MIXlinr.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [5]:
MIXllreg = linear_model.LassoLars(alpha=0.01)
MIXllreg.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [6]:
MIXeregr = ElasticNetCV(cv=5, random_state=17)
MIXeregr.fit(X_train, y_train)


NameError: name 'X_train' is not defined

In [7]:
MIXetreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
MIXetreg.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [74]:
##  Save these four best models

In [75]:
## optional save models
#joblib.dump(MIXlinr, ('../result_models/'+  'covbat_harm_mix_MIXlinr.sav'))
#joblib.dump(MIXllreg, ('../result_models/'+ 'covbat_harmm_mix_MIXllreg.sav'))
#joblib.dump(MIXeregr, ('../result_models/'+ 'covbat_harm_mix_MIXeregr.sav'))
#joblib.dump(MIXetreg, ('../result_models/'+ 'covbat_harm_mix_MIXetreg.sav'))

In [76]:
sabre_ml_matrix = SABRE.drop('participant_id', axis=1)
X_sabre = sabre_ml_matrix.drop('age', axis =1)
X_sabre = X_sabre.values
X_sabre = X_sabre.astype('float')
y_sabre = sabre_ml_matrix['age'].values
y_sabre=y_sabre.astype('float')

In [77]:
X_sabre_test = X_sabre
y_sabre_test = y_sabre

In [3]:
y_sabre_pred = MIXlinr.predict(X_sabre_test)

NameError: name 'MIXlinr' is not defined

In [ ]:
data= [[
    'Linear Reg',
    'unharm_mix_linr0.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXlinr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
linr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_linr_sabre = pd.DataFrame(data)
y_frame_linr_sabre

In [ ]:
y_frame_linr_sabre.to_csv(output_folder + 'y_frame_linr_sabre_covbat_harm.csv')

In [ ]:
y_sabre_pred = MIXllreg.predict(X_sabre_test)

In [ ]:
data= [[
    'Lasso',
    'unharm_mix_lassor0.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXllreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
llreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_llreg_sabre = pd.DataFrame(data)
y_frame_llreg_sabre

In [ ]:
y_frame_llreg_sabre.to_csv(output_folder + 'y_frame_llreg_sabre_covbat_harm.csv')

In [ ]:
y_sabre_pred = MIXeregr.predict(X_sabre_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'harm_mix_elasticregr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXeregr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
eregr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_eregr_sabre = pd.DataFrame(data)
y_frame_eregr_sabre

In [ ]:
y_frame_eregr_sabre.to_csv(output_folder + 'y_frame_eregr_sabre_covbat_harm.csv')

In [ ]:
y_sabre_pred = MIXetreg.predict(X_sabre_test)

In [ ]:
data= [[
    'Extra trees',
    'harm_mix_etreg.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXetreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
etreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_etregr_sabre = pd.DataFrame(data)
y_frame_etregr_sabre

In [ ]:
y_frame_etregr_sabre.to_csv(output_folder + 'y_frame_etregr_sabre_covbat_harm.csv')

In [ ]:
mix_based_covbatharmonized_on_sabre =pd.concat([linr_results_sabre,
                   llreg_results_sabre,
                   eregr_results_sabre,
                  etreg_results_sabre],
                  axis=0)
mix_based_covbatharmonized_on_sabre

In [ ]:
mix_based_covbatharmonized_on_sabre.to_csv(output_folder + 'mix_based_covbatharmonized_on_sabre.csv')

# Running mixed model over Insight46 dataset

## Here we will do an example of running allthe [0] models

In [ ]:
insight_ml_matrix = Insight46.drop('participant_id', axis=1)
X_insight = insight_ml_matrix.drop('age', axis =1)
X_insight = X_insight.values
X_insight = X_insight.astype('float')
y_insight = insight_ml_matrix['age'].values
y_insight= y_insight.astype('float')

In [ ]:
X_insight_test = X_insight
y_insight_test = y_insight

In [ ]:
y_insight_pred = MIXlinr.predict(X_insight_test)

In [ ]:
data= [[
    'Linear Reg',
    'covbatharm_mix_linr0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXlinr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
linr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_linr_insight = pd.DataFrame(data)
y_frame_linr_insight

In [ ]:
y_frame_linr_insight.to_csv(output_folder + 'y_frame_linr_insight_covbat_harm.csv')

In [ ]:
y_insight_pred = MIXllreg.predict(X_insight_test)

In [ ]:
data= [[
    'Lasso',
    'covbatharm_mix_lassor0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXllreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
llreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_llreg_insight = pd.DataFrame(data)
y_frame_llreg_insight

In [ ]:
y_frame_llreg_insight.to_csv(output_folder + 'y_frame_llreg_insight_covbat_harm.csv')

In [ ]:
y_insight_pred = MIXeregr.predict(X_insight_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'covbatharm_mix_elasticregr.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXeregr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
eregr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_eregr_insight = pd.DataFrame(data)
y_frame_eregr_insight

In [ ]:
y_frame_eregr_insight.to_csv(output_folder + 'y_frame_eregr_insight_covbat_harm.csv')

In [ ]:
y_insight_pred = MIXetreg.predict(X_insight_test)

In [ ]:
data= [[
    'Extra trees',
    'covbatharm_mix_etreg.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXetreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
etreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_etreg_insight = pd.DataFrame(data)
y_frame_etreg_insight

In [ ]:
y_frame_etreg_insight.to_csv(output_folder + 'y_frame_etreg_insight_covbat_harm.csv')

In [ ]:
mix_based_covbatharmonized_on_insight =pd.concat([linr_results_insight,
                   llreg_results_insight,
                   eregr_results_insight,
                   etreg_results_insight],
                  axis=0)
mix_based_covbatharmonized_on_insight

In [ ]:
mix_based_covbatharmonized_on_insight.to_csv(output_folder + 'mix_based_covbat_on_insight.csv')